In [1]:
%load_ext Cython

In [2]:
%%cython
# cython: embedsignature=True
# cython: cdivision=True
# cython: wraparound=False
# cython: boundscheck=False
# distutils: language = c++
#
# Cython version of the Navarro & Fuss, 2009 DDM PDF. Based on the following code by Navarro & Fuss:
# http://www.psychocmath.logy.adelaide.edu.au/personalpages/staff/danielnavarro/resources/wfpt.m
#
# This implementation is about 170 times faster than the matlab
# reference version.
#
# Copyleft Thomas Wiecki (thomas_wiecki[at]brown.edu) & Imri Sofer, 2011
# GPLv3

import hddm

include "integrate.pxi"

import scipy.integrate as integrate
from copy import copy
from libcpp cimport bool as bool_t
import numpy as np

cimport numpy as np
cimport cython

from cython.parallel import *

def wiener_like_frlddm_regress(np.ndarray[double, ndim=1] x,
                      np.ndarray[long, ndim=1] response,
                      np.ndarray[double, ndim=1] feedback,
                      np.ndarray[long, ndim=1] split_by,
                      np.ndarray[long, ndim=1] light,
                       np.ndarray[long, ndim=1] stim,
                      double qS, double qL, double alpha, double pos_alpha, double v, 
                      double sv, double a, double z, double zt, double sz, double t,
                      double st, double forg, double surp, double ld, double alpha_stim, double v_stim, double a_stim,
                      double zt_stim, double t_stim, double forg_stim, double surp_stim, double ld_stim,
                      double err, int n_st=10, int n_sz=10, bint use_adaptive=1, double simps_err=1e-8,
                      double p_outlier=0, double w_outlier=0):
    cdef Py_ssize_t size = x.shape[0]
    cdef Py_ssize_t i, j
    cdef Py_ssize_t s_size
    cdef int s
    cdef double p
    cdef double sum_logp = 0
    cdef double wp_outlier = w_outlier * p_outlier
    cdef double alfa
    cdef double pos_alfa
    cdef np.ndarray[double, ndim=1] qs = np.array([qS, qS, qL])
    cdef np.ndarray[double, ndim=1] vs = np.array([0., 0.])
    cdef np.ndarray[double, ndim=1] xs
    cdef np.ndarray[double, ndim=1] feedbacks
    cdef np.ndarray[long, ndim=1] responses
    cdef np.ndarray[long, ndim=1] lights
    cdef np.ndarray[long, ndim=1] unique = np.unique(split_by)
    cdef double z_trial
    cdef double alphaT
    cdef double pe
    
    cdef double alpha0
    cdef double v0
    cdef double a0
    cdef double t0
    cdef double zt0
    cdef double forg0
    cdef double surp0
    cdef double ld0
    
    cdef double e = 2.718281828459

    if pos_alpha==100.00:
        pos_alfa = alpha
    else:
        pos_alfa = pos_alpha

    # unique represent # of conditions
    for j in range(unique.shape[0]):
        s = unique[j]
        # select trials for current condition, identified by the split_by-array
        feedbacks = feedback[split_by == s]
        responses = response[split_by == s]
        lights = light[split_by == s]
        stims = stim[split_by == s]
        xs = x[split_by == s]
        s_size = xs.shape[0]
        qs = np.array([qS, qS, qL])
        
        if stims[0]:
            alpha0 = alpha + alpha_stim
            v0 = v + v_stim
            a0 = a + a_stim
            t0 = t + t_stim
            zt0 = zt + zt_stim
            forg0 = forg + forg_stim
            surp0 = surp + surp_stim
            ld0 = ld + ld_stim
        else:
            alpha0 = alpha
            v0 = v
            a0 = a
            t0 = t
            zt0 = zt
            forg0 = forg
            surp0 = surp
            ld0 = ld
        if forg0 == 100.0:
            forg0 = 0
        else:
            forg0 = (e**forg0) / (1 + e**forg0)
        alpha0 = (e**alpha0) / (1 + e**alpha0)
        if surp0 == 100.0:
            surp0 = 0
        else:
            surp0 = e**surp0
        if ld0 == 100.0:
            ld0 = 0

        # loop through all trials in current condition
        for i in range(s_size):
            vs[0] = qs[0] + (1-lights[i]) * qs[2]
            vs[1] = qs[1] + lights[i] * qs[2]
            if zt==100.0:
                z_trial = z
            else:
                z_trial = z + zt0 * (qs[1] - qs[0])
            z_trial = (e**z_trial) / (1 + e**z_trial)
            p = full_pdf(xs[i], ((vs[1] - vs[0]) * v0) + 2 * (lights[i] - 0.5) * ld0, sv, a0, z_trial,
                         sz, t0, st, err, n_st, n_sz, use_adaptive, simps_err)
            # If one probability = 0, the log sum will be -Inf
            p = p * (1 - p_outlier) + wp_outlier
            if p == 0:
                return -np.inf
            sum_logp += log(p)

            # qs[1] is upper bound, qs[0] is lower bound. feedbacks is reward
            # received on current trial.
            pe = feedbacks[i] - qs[responses[i]]
            alphaT = alpha0 * abs(pe) ** surp0
            qs[responses[i]] = qs[responses[i]] + alphaT * pe
            qs[1-responses[i]] = qs[1-responses[i]] * (1 - forg0)
            if responses[i] == lights[i]:
                pe = feedbacks[i] - qs[2]
                alphaT = alpha0 * abs(pe) ** surp0
                qs[2] = qs[2] + alphaT * pe
            else:
                qs[2] = qs[2] * (1 - forg0)
    return sum_logp

In [3]:
"""
"""

from copy import copy
import numpy as np
import pymc as pm
import wfpt

from kabuki.hierarchical import Knode
from kabuki.utils import stochastic_from_dist
from hddm.models import HDDM


class HDDMfrlReg(HDDM):
    """HDDM model that can be used for two-armed bandit tasks."""

    def __init__(self, *args, **kwargs):
        self.sex_effects = kwargs.pop("sex_effects",[])
        self.biased = kwargs.pop("biased", False)
        self.forgetful = kwargs.pop("forgetful", False)
        self.surprised = kwargs.pop("surprised", False)
        self.non_centered = kwargs.pop("non_centered", False)
        self.light_biased = kwargs.pop("light_biased", False)
        self.dual = kwargs.pop("dual", False)
        self.alpha = kwargs.pop("alpha", True)
        self.wfpt_frl_class = WienerfRL
        for v in self.sex_effects:
                    kwargs['group_only_nodes'].append(v + "_sex")
        super(HDDMfrlReg, self).__init__(*args, **kwargs)
            
    
    def create_sex_nodes(self, knodes):
        for v in self.sex_effects:
            knodes.update(
            self._create_family_normal_normal_hnormal(
                    v + "_sex", value=0, g_mu=0, g_tau=3**-2, std_std=2
                )
            )
        return knodes

    def _create_stochastic_knodes(self, include):
        if "z" in include:
            include.remove("z")
            knodes = super(HDDMfrlReg, self)._create_stochastic_knodes(include)
            knodes.update(
            self._create_family_normal_normal_hnormal(
                    "z", value=0, g_mu=0, g_tau=3**-2, std_std=2
                )
            )
        else:
            knodes = super(HDDMfrlReg, self)._create_stochastic_knodes(include)
        knodes["qS_bottom"] = Knode(
                pm.Beta,
                "qS",
                alpha=1,
                beta=1,
                value=0.33,
                depends=self.depends["qS"],
            )
        knodes["qL_bottom"] = Knode(
                pm.Beta,
                "qL",
                alpha=1,
                beta=1,
                value=0.66,
                depends=self.depends["qL"],
            )
        knodes = self.create_sex_nodes(knodes)
        if self.light_biased:
            knodes.update(
            self._create_family_normal_normal_hnormal(
                    "ld", value=0, g_mu=0, g_tau=3**-2, std_std=2
                )
            )
        if self.biased:
            knodes.update(
            self._create_family_normal_normal_hnormal(
                    "zt", value=0, g_mu=0, g_tau=3**-2, std_std=2
                )
            )
        if self.forgetful:
            knodes.update(
                    self._create_family_normal(
                        "forg",
                        value=-3,
                        g_mu=-2.5,
                        g_tau=3**-2,
                        std_lower=1e-10,
                        std_upper=10,
                        std_value=0.1,
                    )
                )
        if self.surprised:
            knodes.update(
                    self._create_family_normal(
                        "surp",
                        value=0,
                        g_mu=0,
                        g_tau=3**-2,
                        std_lower=1e-10,
                        std_upper=10,
                        std_value=0.1,
                    )
                )
        if self.non_centered:
            print("setting learning rate parameter(s) to be non-centered")
            if self.alpha:
                knodes.update(
                    self._create_family_normal_non_centered(
                        "alpha",
                        value=0,
                        g_mu=0.2,
                        g_tau=3**-2,
                        std_lower=1e-10,
                        std_upper=10,
                        std_value=0.1,
                    )
                )
            if self.dual:
                knodes.update(
                    self._create_family_normal_non_centered(
                        "pos_alpha",
                        value=0,
                        g_mu=0.2,
                        g_tau=3**-2,
                        std_lower=1e-10,
                        std_upper=10,
                        std_value=0.1,
                    )
                )
        else:
            if self.alpha:
                knodes.update(
                    self._create_family_normal(
                        "alpha",
                        value=0,
                        g_mu=0.2,
                        g_tau=3**-2,
                        std_lower=1e-10,
                        std_upper=10,
                        std_value=0.1,
                    )
                )
            if self.dual:
                knodes.update(
                    self._create_family_normal(
                        "pos_alpha",
                        value=0,
                        g_mu=0.2,
                        g_tau=3**-2,
                        std_lower=1e-10,
                        std_upper=10,
                        std_value=0.1,
                    )
                )

        return knodes
    
    def create_sex_parents(self, wfpt_parents, knodes):
        for v in self.sex_effects:
            wfpt_parents[v + "_sex"] = knodes["%s_sex_bottom" % v]
        return wfpt_parents

    def _create_wfpt_parents_dict(self, knodes):
        wfpt_parents = super(HDDMfrlReg, self)._create_wfpt_parents_dict(knodes)
        wfpt_parents = self.create_sex_parents(wfpt_parents, knodes)
        if "z_bottom" in knodes:
            wfpt_parents["z"] = knodes["z_bottom"]
        wfpt_parents["alpha"] = knodes["alpha_bottom"]
        wfpt_parents["qS"] = knodes["qS_bottom"]
        wfpt_parents["qL"] = knodes["qL_bottom"]
        wfpt_parents["pos_alpha"] = knodes["pos_alpha_bottom"] if self.dual else 100.00
        wfpt_parents["zt"] = knodes["zt_bottom"] if self.biased else 100.00
        wfpt_parents["forg"] = knodes["forg_bottom"] if self.forgetful else 100.00
        wfpt_parents["surp"] = knodes["surp_bottom"] if self.surprised else 100.00
        wfpt_parents["ld"] = knodes["ld_bottom"] if self.light_biased else 100.00
        return wfpt_parents

    def _create_wfpt_knode(self, knodes):
        wfpt_parents = self._create_wfpt_parents_dict(knodes)
        return Knode(
            self.wfpt_frl_class,
            "wfpt",
            observed=True,
            col_name=["split_by", "feedback", "response", "light", "rt", "sex"],
            **wfpt_parents
        )

def wienerfRL_like(x, v, alpha, pos_alpha, sv, a, z, zt, sz, t, st, forg, surp, ld, qS, qL, p_outlier=0, alpha_sex=0,v_sex=0,a_sex=0,zt_sex=0,t_sex=0,forg_sex=0,surp_sex=0, ld_sex=0):
        """Log-likelihood for the full DDM using the interpolation method"""
        response = x["response"].values.astype(int)
        feedback = x["feedback"].values.astype(float)
        light = x["light"].values.astype(int)
        split_by = x["split_by"].values.astype(int)
        sex = x["sex"].values.astype(int)
        wiener_params = {
            "err": 1e-4,
            "n_st": 2,
            "n_sz": 2,
            "use_adaptive": 1,
            "simps_err": 1e-3,
            "w_outlier": 0.1,
        }
        return wiener_like_frlddm_regress(
            x["rt"].values.astype(float),
            response,
            feedback,
            split_by,
            light,
            sex,
            qS, qL,
            alpha,
            pos_alpha,
            v,
            sv,
            a,
            z,
            zt,
            sz,
            t,
            st,
            forg,
            surp,
            ld,
            alpha_sex, v_sex, a_sex, zt_sex, t_sex, forg_sex, surp_sex, ld_sex,
            p_outlier=p_outlier,
            **wiener_params
        )


WienerfRL = stochastic_from_dist("wienerfRL", wienerfRL_like)

In [11]:
import os
if not os.path.exists("Models"):
    os.makedirs("Models")

data = hddm.load_csv('hddm_nic.csv')

model_name = "fRLDDM_nic10"

if not os.path.exists("Models/"+model_name):
    os.makedirs("Models/"+model_name)

from p_tqdm import p_map
from functools import partial

def ms1(id, data=None, samples=None, burn=None, thin=None, save_name=None): 

    print('running model%i'%id);
    
    import hddm
    
    mname  = save_name + '_chain_%i.csv'%id
    m = HDDMfrlReg(data, include=["a", "t", "v","st","z"], group_only_nodes=['st', 'z'], depends_on={"v":['sex'],"alpha":['sex'],"a":['sex'],"t":['sex'],"zt":['sex'],"forg":['sex'],"surp":['sex']}, light_biased=False, biased=True, forgetful=True, surprised=True, p_outlier=0.01)
    m.find_starting_values()
    m.sample(samples, burn=burn, thin=thin) # it's neccessary to save the model data
    traces = m.get_traces()
    traces.to_csv(mname)
    
    return m.dic

samples = 2000  # Cavanagh used 30,000 and 10, 000 burn.
burn = 1000    
thin = 1
chains = 4
models2 = p_map(partial(ms1, data=data, samples=samples, burn=burn, thin=thin, save_name="Models/{}/{}".format(model_name,model_name)), range(chains))

running model0
No model attribute --> setting up standard HDDM

  0%|          | 0/4 [00:00<?, ?it/s]


Set model to full_ddm
running model1
No model attribute --> setting up standard HDDM
running model2Set model to full_ddm

No model attribute --> setting up standard HDDM
running model3
No model attribute --> setting up standard HDDMSet model to full_ddm

Set model to full_ddm


/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


                   1%                  ] 22 of 2000 complete in 14.6 sec                  0%                  ] 2 of 2000 complete in 0.6 sec[                  0%                  ] 2 of 2000 complete in 0.7 sec[                  0%                  ] 3 of 2000 complete in 1.1 sec[                  0%                  ] 3 of 2000 complete in 1.3 sec[                  0%                  ] 3 of 2000 complete in 1.3 sec[                  0%                  ] 3 of 2000 complete in 1.5 sec[                  0%                  ] 4 of 2000 complete in 1.7 sec[                  0%                  ] 4 of 2000 complete in 1.9 sec[                  0%                  ] 4 of 2000 complete in 2.0 sec[                  0%                  ] 4 of 2000 complete in 2.3 sec[                  0%                  ] 5 of 2000 complete in 2.4 sec[                  0%                  ] 5 of 2000 complete in 2.6 sec[                  0%                  ] 5 of 2000 complete in 2.7 sec[                  

...

 [-----------------93%---------------   ] 1878 of 2000 complete in 1321.5 sec[-----------------94%---------------   ] 1886 of 2000 complete in 1321.6 sec[-----------------94%---------------   ] 1885 of 2000 complete in 1321.7 sec[-----------------94%---------------   ] 1880 of 2000 complete in 1321.8 sec[-----------------93%---------------   ] 1879 of 2000 complete in 1322.2 sec[-----------------94%---------------   ] 1887 of 2000 complete in 1322.3 sec[-----------------94%---------------   ] 1886 of 2000 complete in 1322.3 sec[-----------------94%---------------   ] 1881 of 2000 complete in 1322.4 sec[-----------------94%---------------   ] 1880 of 2000 complete in 1322.8 sec[-----------------94%---------------   ] 1888 of 2000 complete in 1323.0 sec[-----------------94%---------------   ] 1887 of 2000 complete in 1323.0 sec[-----------------94%---------------   ] 1882 of 2000 complete in 1323.2 sec[-----------------94%---------------   ] 1881 of 2000 complete in 1323.5 sec[----------

 [-----------------95%----------------  ] 1905 of 2000 complete in 1339.9 sec[-----------------95%----------------  ] 1913 of 2000 complete in 1340.2 sec[-----------------95%----------------  ] 1912 of 2000 complete in 1340.2 sec[-----------------95%----------------  ] 1907 of 2000 complete in 1340.6 sec[-----------------95%----------------  ] 1906 of 2000 complete in 1340.7 sec[-----------------95%----------------  ] 1914 of 2000 complete in 1340.9 sec[-----------------95%----------------  ] 1913 of 2000 complete in 1341.0 sec[-----------------95%----------------  ] 1908 of 2000 complete in 1341.3 sec[-----------------95%----------------  ] 1907 of 2000 complete in 1341.4 sec[-----------------95%----------------  ] 1915 of 2000 complete in 1341.6 sec[-----------------95%----------------  ] 1914 of 2000 complete in 1341.6 sec[-----------------95%----------------  ] 1909 of 2000 complete in 1342.0 sec[-----------------95%----------------  ] 1908 of 2000 complete in 1342.1 sec[----------

 [-----------------96%----------------  ] 1933 of 2000 complete in 1358.8 sec[-----------------96%----------------  ] 1932 of 2000 complete in 1358.9 sec[-----------------96%----------------  ] 1939 of 2000 complete in 1359.3 sec[-----------------97%----------------  ] 1941 of 2000 complete in 1359.4 sec[-----------------96%----------------  ] 1933 of 2000 complete in 1359.4 sec[-----------------96%----------------  ] 1934 of 2000 complete in 1359.6 sec[-----------------97%----------------  ] 1940 of 2000 complete in 1359.9 sec[-----------------97%----------------  ] 1942 of 2000 complete in 1360.1 sec[-----------------96%----------------  ] 1935 of 2000 complete in 1360.3 sec[-----------------96%----------------  ] 1934 of 2000 complete in 1360.3 sec[-----------------97%----------------  ] 1941 of 2000 complete in 1360.7 sec[-----------------97%----------------  ] 1943 of 2000 complete in 1360.7 sec[-----------------96%----------------  ] 1935 of 2000 complete in 1360.9 sec[----------

 -----------------98%----------------- ] 1974 of 2000 complete in 1388.4 secc[-----------------98%----------------- ] 1965 of 2000 complete in 1377.9 sec[-----------------98%----------------- ] 1968 of 2000 complete in 1378.1 sec[-----------------98%----------------- ] 1960 of 2000 complete in 1378.3 sec[-----------------98%----------------- ] 1961 of 2000 complete in 1378.5 sec[-----------------98%----------------- ] 1966 of 2000 complete in 1378.6 sec[-----------------98%----------------- ] 1969 of 2000 complete in 1378.8 sec[-----------------98%----------------- ] 1961 of 2000 complete in 1379.0 sec[-----------------98%----------------- ] 1962 of 2000 complete in 1379.2 sec[-----------------98%----------------- ] 1967 of 2000 complete in 1379.3 sec[-----------------98%----------------- ] 1970 of 2000 complete in 1379.6 sec[-----------------98%----------------- ] 1962 of 2000 complete in 1379.6 sec[-----------------98%----------------- ] 1963 of 2000 complete in 1379.9 sec[----------

 [-----------------99%----------------- ] 1986 of 2000 complete in 1396.8 sec[-----------------99%----------------- ] 1995 of 2000 complete in 1396.9 sec[-----------------99%----------------- ] 1993 of 2000 complete in 1397.4 sec[-----------------99%----------------- ] 1988 of 2000 complete in 1397.5 sec[-----------------99%----------------- ] 1987 of 2000 complete in 1397.5 sec[-----------------99%----------------- ] 1996 of 2000 complete in 1397.6 sec[-----------------99%----------------- ] 1994 of 2000 complete in 1398.0 sec[-----------------99%----------------- ] 1989 of 2000 complete in 1398.2 sec[-----------------99%----------------- ] 1988 of 2000 complete in 1398.2 sec[-----------------99%----------------- ] 1997 of 2000 complete in 1398.4 sec[-----------------99%----------------- ] 1995 of 2000 complete in 1398.7 sec[-----------------99%----------------- ] 1990 of 2000 complete in 1399.0 sec[-----------------99%----------------- ] 1989 of 2000 complete in 1398.9 sec[----------

In [12]:
print(models2)

[74514.9823593119, 74512.02373470273, 74509.78885397174, 74511.37621327094]
